In [1]:
import pandas as pd
import numpy as np

In [2]:
reserve_tb = pd.read_csv('./data/reserve.csv', encoding='UTF-8')

In [3]:
production_tb = pd.read_csv('./data/production.csv', encoding='UTF-8')

In [4]:
monthly_index_tb = pd.read_csv('./data/monthly_index.csv', encoding='UTF-8')

## 5. 분할
### 5.1. 모델 검증을 위한 레코드 분할
#### 교차 검증
홀드아웃 검증

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

train_data, test_data, train_target, test_target = \
    train_test_split(production_tb.drop('fault_flg', axis=1),
                     production_tb[['fault_flg']],
                     test_size=0.2)

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
test_target.reset_index(inplace=True, drop=True)

In [6]:
row_no_list = list(range(len(train_target)))
k_fold = KFold(n_splits=4, shuffle=True)

for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    train_cv = train_data.iloc[train_cv_no, :]
    test_cv = train_data.iloc[test_cv_no, :]

### 5.2. 모델 검증을 위한 시간 데이터 분할
학습 기간이 일정한 패턴

 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12
--|--|--|--|--|--|--|--|--|--|--|--
학습|학습|학습|학습|학습|학습|검증|검증|  |  |  |  
  |  |학습|학습|학습|학습|학습|학습|검증|검증|  |  
  |  |  |  |학습|학습|학습|학습|학습|학습|검증|검증

학습 기간이 늘어나는 패턴

 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12
--|--|--|--|--|--|--|--|--|--|--|--
학습|학습|학습|학습|학습|학습|검증|검증|  |  |  |  
학습|학습|학습|학습|학습|학습|학습|학습|검증|검증|  |  
학습|학습|학습|학습|학습|학습|학습|학습|학습|학습|검증|검증


#### 학습 및 검증을 위한 시간 데이터 준비

In [7]:
train_window_start = 1
train_window_end = 24
horizon = 12
skip = 12

monthly_index_tb.sort_values(by='year_month')

while True:
    test_window_end = train_window_end + horizon
    
    print(train_window_start, train_window_end, train_window_end+1, test_window_end)
    
    train = monthly_index_tb[train_window_start:train_window_end]
    test = monthly_index_tb[(train_window_end+1):test_window_end]
    
    if test_window_end >= len(monthly_index_tb.index):
        break
        
    train_window_start += skip
    train_window_end += skip

1 24 25 36
13 36 37 48
25 48 49 60
37 60 61 72
49 72 73 84
61 84 85 96
73 96 97 108
85 108 109 120


## 6. 생성

- 적은 데이터를 늘리는 오버샘플링
- 많은 데이터를 줄이는 언더샘플링
- 위 두 가지를 같이 사용

### 6.1. 언더샘플링

- 4,5장의 방법을 활용해 시행
- 언더샘플링보다는 오버샘플링을 권장

### 6.2. 오버샘플링

- SMOTE

#### 오버샘플링

In [13]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=7)

balance_data, balance_target = \
    sm.fit_sample(production_tb[['length', 'thickness']],
                  production_tb['fault_flg'])

## 7. 전개
데이터 집계 결과를 표 형식으로 변환

### 7.1. 가로 데이터로 변환
레코드 형식 -> 표 형식

In [15]:
pd.pivot_table(
    reserve_tb,
    index='customer_id',
    columns='people_num',
    values='reserve_id',
    aggfunc=lambda x: len(x),
    fill_value=0
)

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1
...,...,...,...,...
c_994,1,0,0,0
c_995,2,2,1,3
c_996,0,4,3,0


### 7.2. 희소 행렬로 변환
희소 행렬: 대부분 요소의 값이 0이고 극히 일부만 값을 가지는 거대한 행렬

In [25]:
from scipy.sparse import csc_matrix

cnt_tb = reserve_tb \
    .groupby(['customer_id', 'people_num'])['reserve_id'].size() \
    .reset_index()
cnt_tb.columns = ['customer_id', 'people_num', 'rsv_cnt']

customer_id = pd.Categorical(cnt_tb['customer_id'])
people_num = pd.Categorical(cnt_tb['people_num'])

result = csc_matrix(
    (cnt_tb['rsv_cnt'], (customer_id.codes, people_num.codes)),
    shape=(len(customer_id.categories), len(people_num.categories))
)
print(result)

  (0, 0)	2
  (2, 0)	2
  (3, 0)	1
  (4, 0)	2
  (6, 0)	3
  (9, 0)	2
  (10, 0)	3
  (11, 0)	4
  (13, 0)	1
  (16, 0)	1
  (18, 0)	1
  (19, 0)	3
  (20, 0)	1
  (21, 0)	1
  (22, 0)	2
  (23, 0)	1
  (24, 0)	4
  (25, 0)	2
  (27, 0)	2
  (29, 0)	2
  (30, 0)	3
  (32, 0)	1
  (36, 0)	1
  (37, 0)	1
  (40, 0)	1
  :	:
  (850, 3)	1
  (852, 3)	1
  (853, 3)	1
  (856, 3)	2
  (857, 3)	1
  (858, 3)	3
  (859, 3)	3
  (862, 3)	1
  (863, 3)	1
  (864, 3)	1
  (866, 3)	1
  (867, 3)	1
  (868, 3)	1
  (869, 3)	3
  (870, 3)	2
  (871, 3)	1
  (875, 3)	1
  (876, 3)	2
  (877, 3)	1
  (878, 3)	1
  (879, 3)	4
  (880, 3)	2
  (881, 3)	3
  (884, 3)	3
  (887, 3)	1
